In [129]:
import pandas as pd
import numpy as np
import sqlite3
import re

conn = sqlite3.connect("profiles.db")
cursor = conn.cursor()
cursor.execute("ATTACH DATABASE 'colleges.db' AS colleges")

In [130]:
df = pd.read_sql_query("SELECT * FROM colleges.university_profiles AS c JOIN profiles AS p ON c.school = p.school",conn)
df_w_admission_rate = df.copy(deep=True)

In [131]:
def get_type_map(df_column):
    return {t:i for i, t in enumerate(set(df_column))}

def convert_sat_to_act(sat):
    if np.isnan(sat):
        return np.nan
    sat_scores = np.array([990, 1060, 1140, 1210, 1270, 1330, 1390, 1450, 1510, 1560, 1620, 1680, 1740, 1800, 1860, 1920, 1980, 2020, 2080, 2140, 2220, 2290, 2380, 2410])
    return np.where(sat < sat_scores)[0][0] + 13
    
def combine_test_scores(x):
    if not np.isnan(x['act']):
        return x['act']/36.0
    sat = np.array([x['sat_m'], x['sat_r'], x['sat_w']])
    return convert_sat_to_act(3*sat[~np.isnan(sat)].mean())/36.0 if len(sat[np.isnan(sat)]) != 3 else np.nan

def convert_string(x):
    try:
        return int(x)
    except:
        return np.nan

def combine_avg_test_scores(x):
    if not np.isnan(convert_string(x['avg_act'])):
        return convert_string(x['avg_act'])/36.0
    sat = np.array([convert_string(x['sat_math']), convert_string(x['sat_reading']), convert_string(x['sat_writing'])])
    return convert_sat_to_act(3*sat[~np.isnan(sat)].mean())/36.0 if len(sat[np.isnan(sat)]) != 3 else np.nan

def map_statuses(x):
    status_map = {"Denied": 0, "Deferred": 0, "Wait-Listed": 0, "Accepted": 2, "Will Attend": 2}
    return status_map[x['status']]

def convert_class_rank(x):
    s = str(x['class_rank'])
    if " of " in s:
        return float(s[:s.index(" of ")])/float(s[s.index(" of ") + 4:])
    if "Top " in s:
        return float(s[4:s.index("%")])/100.0
    if "Bottom " in s:
        return float(s[7:s.index("%")])/100.0
    return np.nan

def convert_instate_tuition(x):
    if "<br>" in str(x['cost_attendance']):
        instate = x['cost_attendance'].split("<br>")[0]
        return instate[instate.index("$")+1:]
    else:
        return x['cost_attendance']

def convert_outstate_tuition(x):
    if "<br>" in str(x['cost_attendance']):
        outstate = x['cost_attendance'].split("<br>")[1]
        return outstate[outstate.index("$")+1:]
    else:
        return x['cost_attendance']

def get_tuition(x):
    return x['in_state_tuition'] if x['state'] == x['hs_state'] else x['out_state_tuition']

def map_hs_types(x):
    hs_type_map = {"Public": 0, "Private": 1, "Parochial": 2, "Home": 3}
    return hs_type_map[x['hs_type']]

def map_ins_types(x):
    ins_type_map = {"Public": 0, "Private": 1, "Private for-profit": 1, "&nbsp;": np.nan}
    return ins_type_map[x['institution_type']]

In [132]:
df = df.loc[:,~df.columns.duplicated()]
df.fillna(value=pd.np.nan, inplace=True)
df['avg_act'] = df['avg_act'].map(lambda x: str(x).strip('-'))
df['test_score'] = df.apply(combine_test_scores, axis=1)
df['avg_test_score'] = df.apply(combine_avg_test_scores, axis=1)
df['class_rank'] = df.apply(convert_class_rank, axis=1)
df['institution_type'] = df.apply(map_ins_types, axis=1)
df['in_state_tuition'] = df.apply(convert_instate_tuition, axis=1)
df['out_state_tuition'] = df.apply(convert_outstate_tuition, axis=1)
#df['tuition'] = df.apply(get_tuition, axis=1)

for col in ['cost_attendance', 'city', 'sat_m', 'sat_r', 'sat_w', 'act', 'sat_math', 'sat_reading', 'sat_writing', 'avg_act', 'gpa_w']:
    df = df.drop(col, axis=1)

df = df.dropna()#subset=['test_score','avg_test_score','gpa_uw','class_rank','hs_state','hs_type','institution_type'])

df['status'] = df.apply(map_statuses, axis=1)
df['hs_type'] = df.apply(map_hs_types, axis=1)
replace_columns = ['hs_state', 'state', 'gender']
for col in replace_columns:
    df[col].replace(get_type_map(df[col]), inplace=True)

In [133]:
print(len(df.index))

44042


In [134]:
whole_df = df

def split_df_on(feature=None):
    split_dfs = {}
    if feature:
        for val in whole_df[feature].unique():
            split_dfs[val] = (whole_df.loc[whole_df[feature]==val])
    else:
        split_dfs = {'all': whole_df}
    return split_dfs
df.sample(n=100)



,school,state,avg_gpa,average_freshman_aid,admission_rate,faculty_total,international_percent,institution_type,female_percentage,year,...,gpa_uw,class_rank,status,eaed,legacy,athlete,test_score,avg_test_score,in_state_tuition,out_state_tuition
66375,"University of California, San Diego",26,4.08,24736,34,1123,19.,0.0,49.3,2014,...,3.47,0.500000,0,0,0,0,0.750000,0.805556,27531,55545
83550,University of Virginia,48,4.27,29009,27,1482,4.4,0.0,54.7,2017,...,3.78,0.250000,0,1,0,0,0.777778,0.861111,31506,62335
39992,Hawaii Pacific University,44,3.50,19107,75,152,10.,1.0,59.8,2016,...,3.20,0.250000,2,0,0,0,0.750000,0.555556,44334,44334
2003,University of Scranton,39,3.49,27728,76,281,1.4,1.0,58.0,2020,...,3.70,0.113208,2,1,0,0,0.888889,0.694444,61028,61028
67150,"University of California, Los Angeles",26,4.36,24405,18,1570,11.,0.0,57.5,2012,...,3.40,0.058974,0,0,0,0,0.694444,0.805556,30485,58499
27931,North Central College,32,3.66,28633,54,151,2.5,1.0,54.4,2018,...,3.96,0.065868,2,0,0,0,0.666667,0.694444,51228,51228
27107,University of Michigan,31,3.84,27050,29,2791,7.1,0.0,49.8,2012,...,4.00,0.002353,2,1,0,0,1.000000,0.805556,29526,62176
56164,University of Kansas,6,3.60,16455,93,1359,5.5,0.0,51.2,2012,...,3.88,0.100000,2,0,0,0,0.833333,0.722222,25090,40858
69852,University of Southern California,26,3.76,52795,16,2133,13.,1.0,51.5,2013,...,4.00,0.002179,2,0,0,0,0.750000,0.833333,70758,70758
2784,Simmons College,29,3.44,32115,60,245,5.3,1.0,99.5,2009,...,3.50,0.250000,2,0,0,0,0.722222,0.666667,59830,59830


In [135]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.linear_model import LogisticRegression

split_on = None
for val, df_s in split_df_on(split_on).items():
    k = 1
    
    df0 = df_s[df.status == 0]
    df2 = df_s[df.status == 2].sample(n = int(k*len(df0)))
    
    df_final = pd.concat([df0, df2])
    df_final_copy = df_final.copy()
    X = df_final.drop('status', axis=1)
    y = df_final['status']
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    X_test_copy = X_test.copy()
    schools = X_test['school']
    X_test = X_test.drop('school', axis=1)
    X_train = X_train.drop('school', axis=1)
    X_test_copy['status'] = y_test
    print(X_test_copy[X_test_copy['school'] == 'Uni'])
    
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    mlp = MLPClassifier(hidden_layer_sizes=(21, 21, 21),max_iter=1000)
    mlp.fit(X_train,y_train)
    predictions = mlp.predict(X_test)
    print(confusion_matrix(y_test,predictions))
    print(classification_report(y_test,predictions))

    logisticRegr = LogisticRegression()
    logisticRegr.fit(X_train, y_train)
    score = logisticRegr.score(X_test, y_test)
    print(f"Logistic Regression Score for {split_on} {val}: " + str(score))



Empty DataFrame
Columns: [school, state, avg_gpa, average_freshman_aid, admission_rate, faculty_total, international_percent, institution_type, female_percentage, year, gender, hs_type, hs_state, gpa_uw, class_rank, eaed, legacy, athlete, test_score, avg_test_score, in_state_tuition, out_state_tuition, status]
Index: []

[0 rows x 23 columns]
[[1797  411]
 [ 467 1720]]
             precision    recall  f1-score   support

          0       0.79      0.81      0.80      2208
          2       0.81      0.79      0.80      2187

avg / total       0.80      0.80      0.80      4395

Logistic Regression Score for None all: 0.7899886234357224


In [136]:
old_x = X_test_copy.copy()
scaler = StandardScaler()
scaler.fit(X_train)
X = scaler.transform(X_test)

In [146]:

from collections import defaultdict
# Experimental Acceptance Rate
e_a_r = defaultdict(lambda: defaultdict(int))
total_applicants = 0
for res, school in zip(mlp.predict(X), schools):
    if res == 0:
        e_a_r[school]['experimental_reject'] += 1
    elif res == 2:
        e_a_r[school]['experimental_accept'] += 1
    e_a_r[school]['total'] += 1
    total_applicants += 1

for school in schools.unique():
    for student in df_final_copy[df_final_copy['school'] == school].iterrows():
        if student[1][15] == 0:
            e_a_r[school]['observed_reject'] += 1
        else:
            e_a_r[school]['observed_accept'] += 1

print(total_applicants == len(schools))


True


In [147]:
e_a_r

defaultdict(<function __main__.<lambda>>,
            {'Adams State University': defaultdict(int,
                         {'experimental_accept': 1,
                          'observed_reject': 1,
                          'total': 1}),
             'Adelphi University': defaultdict(int,
                         {'experimental_accept': 5,
                          'observed_accept': 13,
                          'total': 5}),
             'Agnes Scott College': defaultdict(int,
                         {'experimental_accept': 2,
                          'observed_accept': 7,
                          'observed_reject': 1,
                          'total': 2}),
             'Alabama State University': defaultdict(int,
                         {'experimental_accept': 1,
                          'observed_accept': 1,
                          'observed_reject': 3,
                          'total': 1}),
             'Albany College of Pharmacy and Health Sciences': defaultdict(int,
  

In [154]:
compare_vs_ground_truth = defaultdict(str)

for (index, row), school in zip(old_x.iterrows(), schools):
    accept_rate = None
    if e_a_r[school]['experimental_accept'] != 0 and e_a_r[school]['experimental_reject'] != 0:
        accept_rate = e_a_r[school]['experimental_accept']/e_a_r[school]['total']
    elif e_a_r[school]['experimental_accept'] == 0:
        accept_rate = 0
    elif e_a_r[school]['experimental_reject'] == 0:
        accept_rate = 1
    actual_accept_rate = None
    if e_a_r[school]['observed_accept'] != 0 and e_a_r[school]['observed_reject'] != 0:
        actual_accept_rate = e_a_r[school]['observed_accept']/(e_a_r[school]['observed_reject'] + e_a_r[school]['observed_accept'])
    elif e_a_r[school]['observed_accept'] == 0:
        actual_accept_rate = 0
    elif e_a_r[school]['observed_reject'] == 0:
        actual_accept_rate = 1
    compare_vs_ground_truth[school] = [accept_rate, actual_accept_rate, int(row['admission_rate'])/100]

In [155]:
compare_vs_ground_truth

defaultdict(str,
            {'Adams State University': [1, 0, 0.99],
             'Adelphi University': [1, 1, 0.73],
             'Agnes Scott College': [1, 0.875, 0.66],
             'Alabama State University': [1, 0.25, 0.98],
             'Albany College of Pharmacy and Health Sciences': [1, 0, 0.67],
             'Albany State University': [1, 0.5, 0.5],
             'Albion College': [1, 1, 0.71],
             'Albright College': [0.5, 1, 0.5],
             'Alfred University': [1, 0.75, 0.63],
             'Allegheny College': [1, 0.7, 0.68],
             'Alvernia University': [1, 0.6666666666666666, 0.74],
             'American International College': [1, 1, 0.72],
             'American University': [0.7333333333333333,
              0.5630252100840336,
              0.29],
             'Anderson University': [1, 0.6, 0.79],
             'Appalachian State University': [0.6875, 0.6341463414634146, 0.7],
             'Arcadia University': [1, 1, 0.62],
             'Arizona 

In [156]:
avg_error = 0
error_rates = {}
for (school, admission_stats), (school_2, comparisons) in zip(e_a_r.items(), compare_vs_ground_truth.items()):
    if admission_stats['total'] > 25:
        if school != school_2:
            print("there's a huge mistake somewhere")
        error = abs(comparisons[1]-comparisons[0])
        weight_error = admission_stats['total']/total_applicants
        avg_error += error*weight_error
        error_rates[school] = {'error': error, 'experimental_rate': comparisons[0], 'observed_rate': comparisons[1], 'university_reported': comparisons[2]}
        error_rates[school].update(admission_stats)

In [157]:
avg_error

0.043884294773014

In [158]:
sorted_errors = sorted(error_rates.items(), reverse=False, key=lambda item: item[1]['error'])
flatten_dict = []
for school, stats in sorted_errors:
    stats['school'] = school
    flatten_dict.append(stats)
flatten_dict

[{'error': 0.013151927437641708,
  'experimental_accept': 19,
  'experimental_rate': 0.4222222222222222,
  'experimental_reject': 26,
  'observed_accept': 64,
  'observed_rate': 0.43537414965986393,
  'observed_reject': 83,
  'school': 'Emory University',
  'total': 45,
  'university_reported': 0.22},
 {'error': 0.01339285714285715,
  'experimental_accept': 24,
  'experimental_rate': 0.42857142857142855,
  'experimental_reject': 32,
  'observed_accept': 99,
  'observed_rate': 0.4419642857142857,
  'observed_reject': 125,
  'school': 'Boston University',
  'total': 56,
  'university_reported': 0.25},
 {'error': 0.013600572655690757,
  'experimental_accept': 15,
  'experimental_rate': 0.45454545454545453,
  'experimental_reject': 18,
  'observed_accept': 56,
  'observed_rate': 0.4409448818897638,
  'observed_reject': 71,
  'school': 'Binghamton University',
  'total': 33,
  'university_reported': 0.4},
 {'error': 0.017526455026455,
  'experimental_accept': 20,
  'experimental_rate': 0.74

In [160]:
output_df = pd.DataFrame(flatten_dict, columns=['school', 'error', 'experimental_rate', 'observed_rate', 'university_reported', 'experimental_accept', 'experimental_reject', 'observed_accept', 'observed_reject'])
#output_df.sample(10).sort_values(by=['error'])
output_df

,school,error,experimental_rate,observed_rate,university_reported,experimental_accept,experimental_reject,observed_accept,observed_reject
0,Emory University,0.013152,0.422222,0.435374,0.22,19,26,64,83
1,Boston University,0.013393,0.428571,0.441964,0.25,24,32,99,125
2,Binghamton University,0.013601,0.454545,0.440945,0.40,15,18,56,71
3,University of Vermont,0.017526,0.740741,0.723214,0.67,20,7,81,31
4,Drexel University,0.021524,0.730769,0.752294,0.79,19,7,82,27
5,Villanova University,0.027050,0.294118,0.321168,0.36,10,24,44,93
6,University of Central Florida,0.031100,0.518519,0.549618,0.50,14,13,72,59
7,University of Virginia,0.031166,0.287671,0.256506,0.27,21,52,69,200
8,University of Michigan,0.036688,0.320000,0.356688,0.29,24,51,112,202
9,"California Polytechnic State University, San L...",0.039634,0.375000,0.335366,0.35,15,25,55,109
